In [ ]:
# import necessary libraries

import pandas as pd
import sqlite3
import os

In [3]:
# Establish a connection

conn = sqlite3.connect('/Users/systems/Downloads/chinook.db')
c = conn.cursor()

In [ ]:
# Define a function to run SQL queries and return cleaned-up dataframes
# The function replaces spaces in column names with underscores and capitalizes the first letter of each column name.
# Example: 'Track Name' becomes 'Track_name'

def sq(q):
    
    '''
    Runs the SQL query q on the SQLite connection.
    Returns a dataframe with column names:
        1. no space, an underline(_) is replaced.
        2. Only the first letter in the column name is upper case, the rest is lower case.
    '''
    df_original = pd.read_sql_query(q, conn)
    df_cleaned_up = df_original.rename(columns = lambda x:x.replace(' ','_').capitalize())
    return df_cleaned_up

In [ ]:
# Checl all tables

tables = sq('''
        SELECT name,type
        FROM sqlite_master
        WHERE type='table'
        ''')
tables

,Name,Type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


In [ ]:
# Check the schema of a table

table_name = 'invoice'
table_info = sq(f'''
        PRAGMA table_info({table_name})
        ''')
table_info

,Cid,Name,Type,Notnull,Dflt_value,Pk
0,0,invoice_id,INTEGER,1,None,1
1,1,customer_id,INTEGER,1,None,0
2,2,invoice_date,DATETIME,1,None,0
3,3,billing_address,NVARCHAR(70),0,None,0
4,4,billing_city,NVARCHAR(40),0,None,0
5,5,billing_state,NVARCHAR(40),0,None,0
6,6,billing_country,NVARCHAR(40),0,None,0
7,7,billing_postal_code,NVARCHAR(10),0,None,0
8,8,total,"NUMERIC(10,2)",1,None,0


| column | meaning |   
| ------- | ------- |
| `cid`   | column ID  | 
| `name`   | column Name | 
| `type`   | data type |
| `notnull ` | `1` means the column cannot be NULL
| `Dflt_value` | `None` = no default
|`pk` | primary key |


In [ ]:
# See the datasets

table_entry = sq(f'''
        SELECT * FROM {table_name}
        ''')
table_entry

,Invoice_id,Customer_id,Invoice_date,Billing_address,Billing_city,Billing_state,Billing_country,Billing_postal_code,Total
0,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
1,2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.90
2,3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
3,4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
4,5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83
...,...,...,...,...,...,...,...,...,...
609,610,55,2020-12-21 00:00:00,421 Bourke Street,Sidney,NSW,Australia,2010,6.93
610,611,52,2020-12-27 00:00:00,202 Hoxton Street,London,None,United Kingdom,N1 5LH,1.98
611,612,33,2020-12-27 00:00:00,5112 48 Street,Yellowknife,NT,Canada,X1A 1N6,11.88
612,613,20,2020-12-29 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,8.91


In [ ]:
# See the create statement of all tables

tables = sq('''
            SELECT sql FROM sqlite_master
            WHERE type='table'
            ''')

# Print each create statement

for row in tables['Sql']:
    print(row, '\n')

CREATE TABLE [album]
(
    [album_id] INTEGER PRIMARY KEY NOT NULL,
    [title] NVARCHAR(160)  NOT NULL,
    [artist_id] INTEGER  NOT NULL,
    FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
) 

CREATE TABLE [artist]
(
    [artist_id] INTEGER PRIMARY KEY NOT NULL,
    [name] NVARCHAR(120)
) 

CREATE TABLE [customer]
(
    [customer_id] INTEGER PRIMARY KEY NOT NULL,
    [first_name] NVARCHAR(40)  NOT NULL,
    [last_name] NVARCHAR(20)  NOT NULL,
    [company] NVARCHAR(80),
    [address] NVARCHAR(70),
    [city] NVARCHAR(40),
    [state] NVARCHAR(40),
    [country] NVARCHAR(40),
    [postal_code] NVARCHAR(10),
    [phone] NVARCHAR(24),
    [fax] NVARCHAR(24),
    [email] NVARCHAR(60)  NOT NULL,
    [support_rep_id] INTEGER,
    FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
) 

CREATE TABLE [employee]
(
    [employee_id] INTEGER PRIMARY KEY NOT NULL,
    [last_name] NV

In [ ]:
# Display each dataframe + their information

for i, row in tables.iterrows():
    table = sq(f'''
            SELECT * FROM {row['name']}
            ''')
    display(table.head()) # pretty HTML rendering
    print(table.info()) # 

,Album_id,Title,Artist_id
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Album_id   347 non-null    int64 
 1   Title      347 non-null    object
 2   Artist_id  347 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.3+ KB
None


,Artist_id,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Artist_id  275 non-null    int64 
 1   Name       275 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.4+ KB
None


,Customer_id,First_name,Last_name,Company,Address,City,State,Country,Postal_code,Phone,Fax,Email,Support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Playlist_id  18 non-null     int64 
 1   Name         18 non-null     object
dtypes: int64(1), object(1)
memory usage: 420.0+ bytes
None


,Playlist_id,Track_id
0,1,3402
1,1,3389
2,1,3390
3,1,3391
4,1,3392


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8715 entries, 0 to 8714
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Playlist_id  8715 non-null   int64
 1   Track_id     8715 non-null   int64
dtypes: int64(2)
memory usage: 136.3 KB
None


,Track_id,Name,Album_id,Media_type_id,Genre_id,Composer,Milliseconds,Bytes,Unit_price
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3503 entries, 0 to 3502
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Track_id       3503 non-null   int64  
 1   Name           3503 non-null   object 
 2   Album_id       3503 non-null   int64  
 3   Media_type_id  3503 non-null   int64  
 4   Genre_id       3503 non-null   int64  
 5   Composer       2525 non-null   object 
 6   Milliseconds   3503 non-null   int64  
 7   Bytes          3503 non-null   int64  
 8   Unit_price     3503 non-null   float64
dtypes: float64(1), int64(6), object(2)
memory usage: 246.4+ KB
None


### 1 - what are Customers' fullnames, IDs and countries, who are NOT in the US?

In [ ]:
# Check the columns of a table

customers_table = sq('''
                SELECT * FROM customer AS cs
                ''')
customers_table.columns

Index(['Customer_id', 'First_name', 'Last_name', 'Company', 'Address', 'City',
       'State', 'Country', 'Postal_code', 'Phone', 'Fax', 'Email',
       'Support_rep_id'],
      dtype='object')

In [ ]:
# in SQLite CONCAT raises an error --> the corresponding operator || is used instead.
# to handle NaN values, COALESCE is used, which replaces a NA with an empty atring.
# to ensure first and las names come with thge 1st letter upper case, I used UPPER(SUBSTRING()) and LOWER method.

customers_table_subset = sq('''
                SELECT COALESCE(UPPER(SUBSTRING(First_name,1,1)), '') 
                || COALESCE(LOWER(SUBSTRING(First_name,2,LENGTH(First_name))), '')
                || ' ' 
                || COALESCE(UPPER(SUBSTRING(Last_name,1,1)), '')
                || COALESCE(LOWER(SUBSTRING(Last_name,2,LENGTH(Last_name))), '') AS Name, 
                Customer_id,Country
                FROM customer
                WHERE Country != 'USA'
                ''')
customers_table_subset


#The shorter and easier version would be:
#SELECT First_name||' '||Last_name as Name, country, customer_id
#FROM customer 
#WHERE Country != 'USA' 

,Name,Customer_id,Country
0,Luís Gonçalves,1,Brazil
1,Leonie Köhler,2,Germany
2,François Tremblay,3,Canada
3,Bjørn Hansen,4,Norway
4,František Wichterlová,5,Czech Republic
5,Helena Holý,6,Czech Republic
6,Astrid Gruber,7,Austria
7,Daan Peeters,8,Belgium
8,Kara Nielsen,9,Denmark
9,Eduardo Martins,10,Brazil


### 2 - Showing the invoices of customers from Brazil.
The results should show the customers full name, invoice ID, date of the invoice and billing country

In [ ]:
# Check the columns of a table

invoice_table = sq('''
                SELECT * FROM invoice
                ''')
invoice_table.columns

Index(['Invoice_id', 'Customer_id', 'Invoice_date', 'Billing_address',
       'Billing_city', 'Billing_state', 'Billing_country',
       'Billing_postal_code', 'Total'],
      dtype='object')

In [ ]:
# Check the schema of a table

invoice_table_info = sq('''
        PRAGMA table_info(invoice)
        ''')
invoice_table_info

,Cid,Name,Type,Notnull,Dflt_value,Pk
0,0,invoice_id,INTEGER,1,None,1
1,1,customer_id,INTEGER,1,None,0
2,2,invoice_date,DATETIME,1,None,0
3,3,billing_address,NVARCHAR(70),0,None,0
4,4,billing_city,NVARCHAR(40),0,None,0
5,5,billing_state,NVARCHAR(40),0,None,0
6,6,billing_country,NVARCHAR(40),0,None,0
7,7,billing_postal_code,NVARCHAR(10),0,None,0
8,8,total,"NUMERIC(10,2)",1,None,0


In [ ]:
# Apply inner join on customer_id column from CUSTOMER and INVOICE tables.

customers_and_invoice_table_subset = sq('''
                                        
                SELECT COALESCE(UPPER(SUBSTRING(c.First_name,1,1)), '') 
                || COALESCE(LOWER(SUBSTRING(c.First_name,2,LENGTH(First_name))), '')
                || ' ' 
                || COALESCE(UPPER(SUBSTRING(c.Last_name,1,1)), '')
                || COALESCE(LOWER(SUBSTRING(c.Last_name,2,LENGTH(Last_name))), '') AS Name,
                i.Invoice_id, i.Invoice_date, i.Billing_country
                FROM customer AS c
                JOIN invoice AS i ON c.customer_id = i.customer_id
                WHERE country = 'Brazil'
                
                ''')

customers_and_invoice_table_subset

,Name,Invoice_id,Invoice_date,Billing_country
0,Luís Gonçalves,16,2017-01-26 00:00:00,Brazil
1,Luís Gonçalves,77,2017-05-28 00:00:00,Brazil
2,Luís Gonçalves,149,2017-11-30 00:00:00,Brazil
3,Luís Gonçalves,153,2017-12-14 00:00:00,Brazil
4,Luís Gonçalves,182,2018-02-07 00:00:00,Brazil
...,...,...,...,...
56,Fernanda Ramos,456,2019-12-24 00:00:00,Brazil
57,Fernanda Ramos,498,2020-04-02 00:00:00,Brazil
58,Fernanda Ramos,521,2020-05-27 00:00:00,Brazil
59,Fernanda Ramos,532,2020-06-23 00:00:00,Brazil


### 3 - Showing the Employees with Sales Agent titles 

In [ ]:
# Check the columns of a table

employees_table = sq('''
                SELECT * FROM employee
                ''')
employees_table.columns

Index(['Employee_id', 'Last_name', 'First_name', 'Title', 'Reports_to',
       'Birthdate', 'Hire_date', 'Address', 'City', 'State', 'Country',
       'Postal_code', 'Phone', 'Fax', 'Email'],
      dtype='object')

In [ ]:
# The LIKE command is used in a WHERE clause to search for a specified pattern in a column.

sq('''
   SELECT * FROM employee
   WHERE Title LIKE '%Sales%Agent'
   '''      
)

,Employee_id,Last_name,First_name,Title,Reports_to,Birthdate,Hire_date,Address,City,State,Country,Postal_code,Phone,Fax,Email
0,3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
1,4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
2,5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


### 4 - Showing a unique LIST of billing countries from invoice table

#### Method 1 by `DISTINCT` function

In [ ]:
# Get unique billing countries from invoice table

unique_billing_countries_list = sq('''
    SELECT DISTINCT(Billing_country) FROM invoice
    ''')['Billing_country'].to_list()

unique_billing_countries_list


['USA',
 'Canada',
 'France',
 'Poland',
 'India',
 'Germany',
 'Portugal',
 'Finland',
 'Brazil',
 'Denmark',
 'United Kingdom',
 'Norway',
 'Australia',
 'Hungary',
 'Ireland',
 'Sweden',
 'Spain',
 'Chile',
 'Italy',
 'Czech Republic',
 'Netherlands',
 'Belgium',
 'Austria',
 'Argentina']

#### Method 2 by `GROUP BY` function

In [ ]:
# Count customers per billing country, order by number of customers descending

unique_billing_countries = sq('''
    SELECT Billing_country, COUNT(customer_id) AS Number_of_customers 
    FROM invoice
    GROUP BY Billing_country
    ORDER BY Number_of_customers DESC
    ''')

unique_billing_countries


,Billing_country,Number_of_customers
0,USA,131
1,Canada,76
2,Brazil,61
3,France,50
4,Germany,41
5,Czech Republic,30
6,Portugal,29
7,United Kingdom,28
8,India,21
9,Ireland,13


### 5 - Showing invoices associated with each sales agent
The result should include sales agent full name.

In [38]:
# Join three tables: invoices, customers, and employees
# invoice.customer_id → customer.customer_id   (which customer placed the invoice)
# customer.support_rep_id → employee.employee_id   (which employee supports the customer)


sq(
    '''
    SELECT i.invoice_id, e.First_name || ' ' || e.Last_name AS Sales_agent
    FROM invoice AS i
    LEFT JOIN customer AS c ON i.Customer_id = c.Customer_id
    LEFT JOIN employee AS e ON e.Employee_id = c.Support_rep_id
    '''
    
)

,Invoice_id,Sales_agent
0,16,Jane Peacock
1,77,Jane Peacock
2,149,Jane Peacock
3,153,Jane Peacock
4,182,Jane Peacock
...,...,...
609,121,Jane Peacock
610,155,Jane Peacock
611,220,Jane Peacock
612,599,Jane Peacock


### 6 - Showing invoice total, customer name, country and sale agent for all invoices and customers 

In [24]:
print(f'invoice table columns: {list(invoice_table.columns)}')
print('____________________')
print(f'customer table columns: {list(customers_table.columns)}')
print('____________________')
print(f'employee table columns: {list(employees_table.columns)}')

invoice table columns: ['Invoice_id', 'Customer_id', 'Invoice_date', 'Billing_address', 'Billing_city', 'Billing_state', 'Billing_country', 'Billing_postal_code', 'Total']
____________________
customer table columns: ['Customer_id', 'First_name', 'Last_name', 'Company', 'Address', 'City', 'State', 'Country', 'Postal_code', 'Phone', 'Fax', 'Email', 'Support_rep_id']
____________________
employee table columns: ['Employee_id', 'Last_name', 'First_name', 'Title', 'Reports_to', 'Birthdate', 'Hire_date', 'Address', 'City', 'State', 'Country', 'Postal_code', 'Phone', 'Fax', 'Email']


In [ ]:
# Get customers' full names, countries, total invoice amounts, and their support representatives' full names and titles.


sq('''
   SELECT 
      c.First_name || ' ' || c.Last_name AS Customer_name, 
      c.Country,
      i.Total, 
      e.First_name || ' ' || e.Last_name AS Employee_name,
      e.Title
   FROM 
      Customer AS c
   LEFT JOIN
      invoice AS i ON i.Customer_id = c.Customer_id
   LEFT JOIN
      employee AS e ON e.Employee_id = c.Support_rep_id
   ''')

,Customer_name,Country,Total,Employee_name,Title
0,Luís Gonçalves,Brazil,8.91,Jane Peacock,Sales Support Agent
1,Luís Gonçalves,Brazil,5.94,Jane Peacock,Sales Support Agent
2,Luís Gonçalves,Brazil,8.91,Jane Peacock,Sales Support Agent
3,Luís Gonçalves,Brazil,13.86,Jane Peacock,Sales Support Agent
4,Luís Gonçalves,Brazil,5.94,Jane Peacock,Sales Support Agent
...,...,...,...,...,...
609,Puja Srivastava,India,8.91,Jane Peacock,Sales Support Agent
610,Puja Srivastava,India,8.91,Jane Peacock,Sales Support Agent
611,Puja Srivastava,India,8.91,Jane Peacock,Sales Support Agent
612,Puja Srivastava,India,7.92,Jane Peacock,Sales Support Agent


### 7 - Number of invoices and total sales in 2019 and 2020

Method 1 by `GROUP BY` and `HAVING`

In [ ]:
# strftime(format, timestring) ---> %m, %m%Y, ...

sq('''
    SELECT
        COUNT(i.Invoice_id) AS Number_of_invoices,
        SUM(i.Total) AS Total_sale,
        STRFTIME('%Y', i.Invoice_date) AS Year
    FROM
        invoice AS i
    GROUP BY
        STRFTIME('%Y', i.Invoice_date)
    HAVING
        STRFTIME('%Y', i.Invoice_date) = '2019' 
        OR 
        STRFTIME('%Y', i.Invoice_date) = '2020'
    ''')

,Number_of_invoices,Total_sale,Year
0,159,1221.66,2019
1,150,1138.50,2020


Method 2 by `subquery` and `UNION ALL`

In [ ]:
# Alternatively, using UNION ALL
# to combine the results of two or more SELECT statements.
# Each SELECT statement within the UNION ALL must have the same number of columns in the result sets with similar data types.

sq('''
    SELECT
        COUNT(i.Invoice_id) AS Number_of_invoices,
        SUM(i.Total) AS Total_sale,
        STRFTIME('%Y', i.Invoice_date) AS Year
    FROM
        invoice AS i
    WHERE
        STRFTIME('%Y', i.Invoice_date) = '2019'
        
    UNION ALL
    
    SELECT
        COUNT(i.Invoice_id) AS Number_of_invoices,
        SUM(i.Total) AS Total_sale,
        STRFTIME('%Y', i.Invoice_date) AS Year
    FROM
        invoice AS i
    WHERE
        STRFTIME('%Y', i.Invoice_date) = '2020'
    ''')

,Number_of_invoices,Total_sale,Year
0,159,1221.66,2019
1,150,1138.50,2020


In [ ]:
# Check the columns of a table

invoice_line_table = sq('''
   SELECT * FROM invoice_line
   ''')
invoice_line_table

,Invoice_line_id,Invoice_id,Track_id,Unit_price,Quantity
0,1,1,1158,0.99,1
1,2,1,1159,0.99,1
2,3,1,1160,0.99,1
3,4,1,1161,0.99,1
4,5,1,1162,0.99,1
...,...,...,...,...,...
4752,4753,614,2659,0.99,1
4753,4754,614,2660,0.99,1
4754,4755,614,2661,0.99,1
4755,4756,614,2662,0.99,1


#### 7 - 1 - Who are the top 3 agents with the highets sales in 2019?

In [ ]:
# Find the top 5 employees (sales agents) who generated the highest total sales in 2019.
# Join the employee, customer, and invoice tables.
# employee.employee_id → customer.support_rep_id   (which employee supports the customer)
# customer.customer_id → invoice.customer_id   (which customer placed the invoice)
# Filter for invoices from the year 2019.
# Group the results by employee to aggregate their sales.
# Order the results by total sales in descending order.
# Limit the output to the top 5 employees.

sq('''
   SELECT
        e.First_name || ' ' || e.Last_name AS Employee_name,
        SUM(i.Total) AS Total_sale,
        COUNT(i.Invoice_id) AS Number_of_invoices
     FROM
          employee AS e
     JOIN
          customer AS c ON e.Employee_id = c.Support_rep_id
     JOIN
          invoice AS i ON c.Customer_id = i.Customer_id
     WHERE
          STRFTIME('%Y', i.Invoice_date) = '2019'
     GROUP BY
          e.Employee_id
     ORDER BY
          Total_sale DESC
     LIMIT 5
   
   ''')

,Employee_name,Total_sale,Number_of_invoices
0,Steve Johnson,437.58,56
1,Margaret Park,400.95,54
2,Jane Peacock,383.13,49


#### 7 - 2 - who are top 3 agents with highets sales' revenue overall?

In [ ]:
# Find the top 5 employees (sales agents) who generated the highest total sales overall (not limited to a specific year).
# Join the employee, customer, and invoice tables.
# employee.employee_id → customer.support_rep_id   (which employee supports the customer)
# customer.customer_id → invoice.customer_id   (which customer placed the invoice)
# Group the results by employee to aggregate their sales.
# Order the results by total sales in descending order.
# Limit the output to the top 5 employees.

sq('''
   SELECT
        e.First_name || ' ' || e.Last_name AS Employee_name,
        SUM(i.Total) AS Total_sale,
        COUNT(i.Invoice_id) AS Number_of_invoices
     FROM
          employee AS e
     JOIN
          customer AS c ON e.Employee_id = c.Support_rep_id
     JOIN
          invoice AS i ON c.Customer_id = i.Customer_id
     GROUP BY
          e.Employee_id
     ORDER BY
          Total_sale DESC
     LIMIT 5
   
   ''')

,Employee_name,Total_sale,Number_of_invoices
0,Jane Peacock,1731.51,212
1,Margaret Park,1584.00,214
2,Steve Johnson,1393.92,188


### 8 - Number of line items for invoice ID 37

In [ ]:
# Calculate the total quantity of items purchased in invoice ID 37 from the invoice_line table.
# invoice_line.invoice_id → invoice.invoice_id   (which invoice contains the line items)
# Filter for invoice ID 37.
# Group by Invoice_id to aggregate the quantities.
# Order by Invoice_id.

sq('''
   SELECT 
        Invoice_id, SUM(Quantity) AS Quantity
    FROM
        invoice_line
    WHERE
        Invoice_id = 37
   ''')

,Invoice_id,Quantity
0,37,10


### 9 - Number of line items for invoice IDs

In [ ]:
# Calculate the total quantity of items purchased per invoice from the invoice_line table.
# invoice_line.invoice_id → invoice.invoice_id   (which invoice contains the line items)
# Group by Invoice_id to aggregate the quantities.
# Order by Invoice_id.

sq('''
   SELECT 
        Invoice_id, SUM(Quantity) AS Quantity
    FROM
        invoice_line
    GROUP BY
        Invoice_id
           ''')

,Invoice_id,Quantity
0,1,16
1,2,10
2,3,2
3,4,8
4,5,17
...,...,...
609,610,7
610,611,2
611,612,12
612,613,9


### 10 - Track names with invoice line

In [ ]:
# Check the columns of a table

track_table = sq('''
   SELECT * FROM track
   ''')
track_table

# Primary key = Track_id
# Table -- > invoice_line ---> primary key = Invoice_line_id, foreign key = Track_id

,Track_id,Name,Album_id,Media_type_id,Genre_id,Composer,Milliseconds,Bytes,Unit_price
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


In [ ]:
# Join track and invoice_line tables on Track_id to get track names along with their invoice line details.
# track.track_id → invoice_line.track_id   (which track is in which invoice line)
# Join on Track_id

sq('''
   SELECT 
      t.Name, il.*
   FROM
      invoice_line AS il
   JOIN
      track AS t ON t.Track_id = il.Track_id
        
   ''')

,Name,Invoice_line_id,Invoice_id,Track_id,Unit_price,Quantity
0,Right Next Door to Hell,1,1,1158,0.99,1
1,Dust N' Bones,2,1,1159,0.99,1
2,Live and Let Die,3,1,1160,0.99,1
3,Don't Cry (Original),4,1,1161,0.99,1
4,Perfect Crime,5,1,1162,0.99,1
...,...,...,...,...,...,...
4752,Every Breath You Take,4753,614,2659,0.99,1
4753,King Of Pain,4754,614,2660,0.99,1
4754,Wrapped Around Your Finger,4755,614,2661,0.99,1
4755,Don't Stand So Close to Me '86,4756,614,2662,0.99,1


### 11 - The purchased track name AND artist name with each invoice line item

In [ ]:
# Join track, album, artist, and invoice_line tables to get track names, artist names, and invoice line details.
# track.track_id → invoice_line.track_id   (which track is in which invoice line)
# track.album_id → album.album_id   (which album the track belongs to)
# album.artist_id → artist.artist_id   (which artist created the album)
# Join on Track_id, Album_id, and Artist_id

sq('''
   SELECT
        t.Name AS Track_name,
        ar.Name AS Artist_name,
        il.Invoice_id,
        a.Artist_id AS Artist_id,
        il.*
     FROM
          invoice_line AS il
     LEFT JOIN
          track AS t ON t.Track_id = il.Track_id
     LEFT JOIN
          album AS a ON a.Album_id = t.Album_id
     LEFT JOIN 
          artist AS ar ON ar.Artist_id = a.Artist_id
   
   ''')

,Track_name,Artist_name,Invoice_id,Artist_id,Invoice_line_id,Invoice_id,Track_id,Unit_price,Quantity
0,Right Next Door to Hell,Guns N' Roses,1,88,1,1,1158,0.99,1
1,Dust N' Bones,Guns N' Roses,1,88,2,1,1159,0.99,1
2,Live and Let Die,Guns N' Roses,1,88,3,1,1160,0.99,1
3,Don't Cry (Original),Guns N' Roses,1,88,4,1,1161,0.99,1
4,Perfect Crime,Guns N' Roses,1,88,5,1,1162,0.99,1
...,...,...,...,...,...,...,...,...,...
4752,Every Breath You Take,The Police,614,141,4753,614,2659,0.99,1
4753,King Of Pain,The Police,614,141,4754,614,2660,0.99,1
4754,Wrapped Around Your Finger,The Police,614,141,4755,614,2661,0.99,1
4755,Don't Stand So Close to Me '86,The Police,614,141,4756,614,2662,0.99,1


### 12 - calculate the revenue for each artist

In [ ]:
# Calculate the total revenue generated by each artist from the invoice_line table.
# Join track, album, artist, and invoice_line tables.
# track.track_id → invoice_line.track_id   (which track is in which invoice line)
# track.album_id → album.album_id   (which album the track belongs to)
# album.artist_id → artist.artist_id   (which artist created the album)
# Join on Track_id, Album_id, and Artist_id
# Total revenue per artist = SUM(Unit_price * Quantity)
# Group by Artist_name to aggregate the total revenue.
# Order by Total_revenue in descending order.

sq('''
   WITH Price_quantity_table AS
     
    (
        SELECT
            ar.Name AS Artist_name,
            il.Unit_price,
            il.Quantity
        FROM
            invoice_line AS il
        LEFT JOIN
            track AS t ON t.Track_id = il.Track_id
        LEFT JOIN
            album AS a ON a.Album_id = t.Album_id
        LEFT JOIN 
            artist AS ar ON ar.Artist_id = a.Artist_id
        )
        
    SELECT 
        Artist_name,
        SUM(Unit_price * Quantity) AS Total_revenue
    FROM
        Price_quantity_table
    GROUP BY
        Artist_name
    ORDER BY
        Total_revenue DESC;
    
   ''')

,Artist_name,Total_revenue
0,Queen,190.08
1,Jimi Hendrix,185.13
2,Red Hot Chili Peppers,128.70
3,Nirvana,128.70
4,Pearl Jam,127.71
...,...,...
125,Falamansa,0.99
126,Dread Zeppelin,0.99
127,Berliner Philharmoniker & Herbert Von Karajan,0.99
128,Battlestar Galactica,0.99


### 13 - Number of invoices per country

In [ ]:
# Find the top 5 countries with the highest number of invoices from the invoice table.
# Group by Billing_country to count the number of invoices per country.
# Order by the count of invoices in descending order.
# Limit the output to the top 5 countries.

sq('''
   SELECT
        c.Country,
        COUNT(i.Invoice_id) AS Quantity_invoices
    FROM
        customer AS c
    JOIN
        invoice AS i ON i.Customer_id = c.Customer_id
    GROUP BY
        Country
    ORDER BY
        Quantity_invoices DESC
    LIMIT
        5;
   
   ''')

,Country,Quantity_invoices
0,USA,131
1,Canada,76
2,Brazil,61
3,France,50
4,Germany,41


### The total number of tracks in each playlist. 
The Playlist name should be included on the resultant table

In [ ]:
# Check the columns of a table

playlist_track_table = sq('''
   SELECT * FROM playlist_track
   ''')
playlist_track_table

,Playlist_id,Track_id
0,1,3402
1,1,3389
2,1,3390
3,1,3391
4,1,3392
...,...,...
8710,17,2094
8711,17,2095
8712,17,2096
8713,17,3290


In [ ]:
# Check the columns of a table

track_table = sq('''
   SELECT * FROM track
   ''')
track_table

,Track_id,Name,Album_id,Media_type_id,Genre_id,Composer,Milliseconds,Bytes,Unit_price
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


In [ ]:
# Check the columns of a table

playlist_table = sq('''
   SELECT * FROM playlist
   ''')
playlist_table

,Playlist_id,Name
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music
5,6,Audiobooks
6,7,Movies
7,8,Music
8,9,Music Videos
9,10,TV Shows


In [ ]:
# Find the number of tracks in each playlist from the playlist_track and track tables.
# playlist.playlist_id → playlist_track.playlist_id   (which playlist contains which tracks)
# playlist_track.track_id → track.track_id   (which track is in which playlist)
# Join on Playlist_id and Track_id

sq('''
   SELECT 
        p.NAME AS Playlist_name,
        COUNT(t.Track_id) AS Number_of_tracks
   FROM
        playlist AS p
   JOIN
        playlist_track AS pt ON p.Playlist_id = pt.Playlist_id
   JOIN
        track AS t ON t.Track_id = pt.Track_id
   GROUP BY
        p.Playlist_id
   ORDER BY
        Number_of_tracks DESC  
   
   
   ''')

,Playlist_name,Number_of_tracks
0,Music,3290
1,Music,3290
2,90’s Music,1477
3,TV Shows,213
4,TV Shows,213
5,Classical,75
6,Brazilian Music,39
7,Heavy Metal Classic,26
8,Classical 101 - The Basics,25
9,Classical 101 - Next Steps,25
